In [1]:
import submitit

import torch
import random
import numpy as np

import pickle
import itertools
import argparse
import logging
import os
import pathlib
import time
import json
import math
import matplotlib.pyplot as plt
from torch.utils import data
from fvcore.common.config import CfgNode
from submitit.core.utils import FailedJobError
from itertools import cycle
from time import sleep
from multi_objective.main import main, get_config
from plotting.plotting import load_data, plot_row

In [2]:
executor = submitit.AutoExecutor(folder="tmp/submitit")
seeds = [0, 1, 2, 3, 4]

In [3]:
executor.update_parameters(timeout_min=20, slurm_partition="alldlc_gpu-rtx2080", name='results', gpus_per_node=1)
executor.update_parameters(slurm_array_parallelism=100)

In [4]:
def percent_finished(jobs):
    if len(jobs):
        return sum(job.done() for job in jobs) / len(jobs)

def execute(config, seeds):
    cfg = config.clone()
    cfg.eval_every = 1   # early stopping
    cfg.test_eval_every = 1  # generate test results
    
    cfgs = []
    for seed in seeds:
        cfg = cfg.clone()
        cfg.merge_from_list(['seed', seed])
        cfgs.append(cfg)
    
    tags = [f"result_{s :2d}" for s in seeds]
    
    # func, rank, world_size, cfg, tag
    return executor.map_array(main, cycle([0]), cycle([1]), cfgs, tags)

In [5]:
results = load_data(datasets=['multi_mnist', 'multi_fashion', 'multi_fashion_mnist'])

loaded data for multi_mnist
loaded data for multi_fashion
loaded data for multi_fashion_mnist


In [6]:
plot_row(results, datasets=['multi_mnist', 'multi_fashion', 'multi_fashion_mnist'], prefix='baselines')

cosmos
uniform
phn
mgda
cosmos
uniform
phn
mgda
cosmos
uniform
phn
mgda
success. See baselines_multi_mnist_multi_fashion_multi_fashion_mnist.pdf


#### cosmos

In [7]:
jobs = execute(get_config('configs/multi_mnist/cosmos.yaml'), seeds)

In [8]:
jobs = execute(get_config('configs/multi_fashion/cosmos.yaml'), seeds)

In [9]:
jobs = execute(get_config('configs/multi_fashion_mnist/cosmos.yaml'), seeds)

#### mgda

In [10]:
jobs = execute(get_config('configs/multi_mnist/mgda.yaml'), seeds)

In [11]:
jobs = execute(get_config('configs/multi_fashion/mgda.yaml'), seeds)

In [12]:
jobs = execute(get_config('configs/multi_fashion_mnist/mgda.yaml'), seeds)

#### phn

In [13]:
jobs = execute(get_config('configs/multi_mnist/phn.yaml'), seeds)

In [14]:
jobs = execute(get_config('configs/multi_fashion/phn.yaml'), seeds)

In [15]:
jobs = execute(get_config('configs/multi_fashion_mnist/phn.yaml'), seeds)

#### pmtl todo

#### single task

In [16]:
jobs = execute(get_config('configs/multi_mnist/single_task.yaml'), seeds)

In [17]:
jobs = execute(get_config('configs/multi_fashion/single_task.yaml'), seeds)

In [18]:
jobs = execute(get_config('configs/multi_fashion_mnist/single_task.yaml'), seeds)

#### uniform

In [19]:
jobs = execute(get_config('configs/multi_mnist/uniform.yaml'), seeds)

In [20]:
jobs = execute(get_config('configs/multi_fashion/uniform.yaml'), seeds)

In [21]:
jobs = execute(get_config('configs/multi_fashion_mnist/uniform.yaml'), seeds)